In [104]:
!pip install pygame

In [105]:
import pygame
import random
import numpy as np

In [106]:
WIDTH=300
HEIGHT=300
GRID_SIZE=5
CELL_SIZE=WIDTH//GRID_SIZE
FPS=5

In [107]:
UP=(0,-1)
DOWN=(0,1)
LEFT=(-1,0)
RIGHT=(1,0)
ACTIONS=[UP,DOWN,LEFT,RIGHT]

In [108]:
pygame.font.init()
win=pygame.display.set_mode((WIDTH,HEIGHT))
pygame.display.set_caption("Snake Q-learning-Small Grid")
font=pygame.font.SysFont(None,24)

In [109]:
class Food:
  def __init__(self):
    self.position=(random.randint(0,GRID_SIZE-1),random.randint(0,GRID_SIZE-1))

  def randomize(self):
    self.position=(random.randint(0,GRID_SIZE-1),random.randint(0,GRID_SIZE-1))

In [110]:
class Snake:
  def __init__(self):
    self.position=[(2,2)]
    self.direction=random.choice(ACTIONS)

  def move(self,action=None):
    self.direction=action
    head_x,head_y=self.position[0]
    dir_x,dir_y=self.direction
    new_head=(head_x+dir_x,head_y+dir_y)
    self.position=[new_head]+self.position[:-1]

  def grow(self):
    tail=self.position[-1]
    self.position.append(tail)

  def collision(self):
    head=self.position[0]
    return(
        head in self.position[1:] or
        head[0]<0 or head[0]>=GRID_SIZE or
        head[1]<0 or head[1]>=GRID_SIZE
    )

In [111]:
q_table=np.zeros((GRID_SIZE,GRID_SIZE,4))

In [112]:
alpha=0.1 #learning rate
gamma=0.9 #discount factor
epsilon=0.2 #exploration rate
num_episodes=50

In [113]:
def get_state(snake):
  return snake.position[0]

In [114]:
def get_reward(snake, food):
    if snake.position[0] == food.position:
        return 10
    elif snake.collision():
        return -100
    else:
        return -1

In [11]:
# Define colors (move these outside the loop if they are used elsewhere)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

WIDTH=300
HEIGHT=300
GRID_SIZE=5
CELL_SIZE=WIDTH//GRID_SIZE
FPS=5

alpha=0.1 #learning rate
gamma=0.9 #discount factor
epsilon=0.2 #exploration rate
num_episodes=50 # Define num_episodes here

max_steps_per_episode = 1000 # Define a maximum number of steps per episode

import random
import numpy as np
import pygame

UP=(0,-1)
DOWN=(0,1)
LEFT=(-1,0)
RIGHT=(1,0)
ACTIONS=[UP,DOWN,LEFT,RIGHT]

class Food:
  def __init__(self):
    self.position=(random.randint(0,GRID_SIZE-1),random.randint(0,GRID_SIZE-1))

  def randomize(self):
    self.position=(random.randint(0,GRID_SIZE-1),random.randint(0,GRID_SIZE-1))

class Snake:
  def __init__(self):
    self.position=[(2,2)]
    self.direction=random.choice(ACTIONS)

  def move(self,action=None):
    self.direction=action
    head_x,head_y=self.position[0]
    dir_x,dir_y=self.direction
    new_head=(head_x+dir_x,head_y+dir_y)
    self.position=[new_head]+self.position[:-1]

  def grow(self):
    tail=self.position[-1]
    self.position.append(tail)

  def collision(self):
    head=self.position[0]
    return(
        head in self.position[1:] or
        head[0]<0 or head[0]>=GRID_SIZE or
        head[1]<0 or head[1]>=GRID_SIZE
    )

def get_state(snake):
  return snake.position[0]

def get_reward(snake, food):
    if snake.position[0] == food.position:
        return 10
    elif snake.collision():
        return -100
    else:
        return -1


pygame.init()
win=pygame.display.set_mode((WIDTH,HEIGHT))
pygame.display.set_caption("Snake Q-learning-Small Grid")
font=pygame.font.SysFont(None,24)

q_table=np.zeros((GRID_SIZE,GRID_SIZE,4))


for episode in range(1, num_episodes + 1):
    snake_instance = Snake()
    food = Food()
    done = False
    total_reward = 0
    steps = 0

    clock = pygame.time.Clock()

    while not done and steps < max_steps_per_episode:
        clock.tick(FPS)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()

        current_state = get_state(snake_instance)

        if random.uniform(0, 1) < epsilon: # Use epsilon instead of episode for exploration rate
            action_idx = random.randint(0, 3)
        else:
            # Ensure state indices are within bounds before accessing q_table
            state_x, state_y = current_state
            if 0 <= state_x < GRID_SIZE and 0 <= state_y < GRID_SIZE:
                action_idx = np.argmax(q_table[state_x, state_y])
            else:
                # If state is out of bounds, choose a random action or end the episode
                # For now, let's end the episode on collision, handled after move
                action_idx = random.randint(0, 3) # Choose random action as a fallback


        action = ACTIONS[action_idx]
        snake_instance.move(action)


        # Check for collision after moving
        if snake_instance.collision():
            reward = -100
            done = True
        else:
            reward = get_reward(snake_instance, food)
            # If food is eaten, grow the snake and randomize food position
            if snake_instance.position[0] == food.position:
                snake_instance.grow()
                food.randomize()


        new_state = get_state(snake_instance)
        total_reward += reward
        steps += 1

        # Q-learning update
        if 0 <= current_state[0] < GRID_SIZE and 0 <= current_state[1] < GRID_SIZE and \
           0 <= new_state[0] < GRID_SIZE and 0 <= new_state[1] < GRID_SIZE:
             old_value = q_table[current_state[0], current_state[1], action_idx]
             next_max = np.max(q_table[new_state[0], new_state[1]]) if not done else 0
             new_value = old_value + alpha * (reward + gamma * next_max - old_value)
             q_table[current_state[0], current_state[1], action_idx] = new_value


        # Rendering
        win.fill(WHITE) # White background
        # Draw grid lines
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                pygame.draw.rect(win, BLACK, (i * CELL_SIZE, j * CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)
        # Draw snake
        for segment in snake_instance.position:
             pygame.draw.rect(win,GREEN,(segment[0]*CELL_SIZE,segment[1]*CELL_SIZE,CELL_SIZE,CELL_SIZE))
        # Draw food
        pygame.draw.rect(win,RED,(food.position[0]*CELL_SIZE,food.position[1]*CELL_SIZE,CELL_SIZE,CELL_SIZE))
        # Update display
        pygame.display.update()


    print(f"Episode {episode}: Total Reward = {total_reward}")

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
Episode 1: Total Reward = -103
Episode 2: Total Reward = -108
Episode 3: Total Reward = -112
Episode 4: Total Reward = -104
Episode 5: Total Reward = -108
Episode 6: Total Reward = -116
Episode 7: Total Reward = -96
Episode 8: Total Reward = -104
Episode 9: Total Reward = -102
Episode 10: Total Reward = -106
Episode 11: Total Reward = -104
Episode 12: Total Reward = -98
Episode 13: Total Reward = -102
Episode 14: Total Reward = -103
Episode 15: Total Reward = -107
Episode 16: Total Reward = -107
Episode 17: Total Reward = -98
Episode 18: Total Reward = -106
Episode 19: Total Reward = -103
Episode 20: Total Reward = -103
Episode 21: Total Reward = -81
Episode 22: Total Reward = -92
Episode 23: Total Reward = -104
Episode 24: Total Reward = -105
Episode 25: Total Reward = -102
Episode 26: Total Reward = -102
Episode 27: Total Reward = -106
Episode 28: Total Reward = -102
Epis